In [1]:
import time
import random
import json
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
expansions = [  'Duelist-Saga', 'Starter-Deck-Link-Strike', 'Code-of-the-Duelist', 'Legendary-Dragon-Decks', 
                'Circuit-Break', 'Structure-Deck-Cyberse-Link', 'Lost-Art-Promos', 'Legendary-Duelists-Ancient-Millennium',
                'Star-Pack-VRAINS', 'Flames-of-Destruction', 'Dark-Saviors', 'Battles-of-Legend-Relentless-Revenge', 
                'Structure-Deck-Powercode-Link', 'Soul-Fusion', 'Dark-Neostorm', 'Battles-of-Legend-Heros-Revenge', 'Premium-Gold',
                'Eternity-Code', 'Legendary-Duelists-Season-1', 'Battles-of-Legend-Armageddon', 'Legendary-Hero-Decks', 'Fusion-Enforcers'
                '2020-Tin-of-Lost-Memories-Mega-Pack', 'Legendary-Duelists-Rage-of-Ra', 'OTS-Tournament-Pack-14', 'Premium-Gold-Infinite-Gold',
                'Phantom-Rage', 'Maximum-Gold', 'Genesis-Impact', 'Legendary-Duelists-Season-2', 'Blazing-Vortex', 'Savage-Strike',
                'Ancient-Guardians', 'Lightning-Overdrive', 'Demo-Pack', 'Structure-Deck-Shaddoll-Showdown', 'Secret-Slayers',
                'Kings-Court', 'Egyptian-God-Deck-Slifer-the-Sky-Dragon', 'Egyptian-God-Deck-Obelisk-the-Tormentor', 
                'Burst-of-Destiny', 'Dawn-of-Majesty', 'Structure-Deck-Cyber-Strike', 'Battle-of-Chaos', 'The-Grand-Creators', 'Structure-Deck-Synchron-Extreme',
                'Maximum-Gold-El-Dorado', 'Brothers-of-Legend', 'Hidden-Arsenal-Chapter-1', 'Speed-Duel-GX-Duel-Academy-Box', 
                'Structure-Deck-Albaz-Strike', '2022-Ghosts-From-the-Past', 'Dimension-Force', 'Structure-Deck-Legend-of-the-Crystal-Beasts',
                'Legendary-Duelists-Season-3', 'Power-of-the-Elements', 'Tactical-Masters', 'Darkwing-Blast', 'Magnificent-Mavens',
                'Battles-of-Legend-Crystal-Revenge', 'Amazing-Defenders', 'Photon-Hypernova', 'Structure-Deck-Beware-of-Traptrix', 
                'Maze-of-Memories', 'Cyberstorm-Access', 'Wild-Survivors', 'Battles-of-Legend-Monstrous-Revenge', 'Duelist-Nexus', 
                'Legendary-Duelists-Soulburning-Volcano', '25th-Anniversary-Tin-Dueling-Heroes-Mega-Pack', 'Age-of-Overlord', 
                '25th-Anniversary-Rarity-Collection', 'Phantom-Nightmare', 'Legacy-of-Destruction', 'Duel-Devastator', 'Duel-Overload', 
                'Rise-of-the-Duelist', 'Legendary-Duelists-Synchro-Storm', 'Maximum-Crisis', 'Legendary-Collection-4-Mega-Pack', 
                'Starter-Deck-Codebreaker', 'Starter-Deck-Kaiba-Reloaded', 'Extreme-Force', 'Cybernetic-Horizon', 'Legendary-Duelists-White-Dragon-Abyss',
                'Legendary-Duelists-Sisters-of-the-Rose', 'Legendary-Duelists-Immortal-Destiny', 'Legendary-Duelists-Magical-Hero', 
                'Legendary-Duelists-9', 'OTS-Tournament-Pack-14', 'OTS-Tournament-Pack-18', 'OTS-Tournament-Pack-19', 'OTS-Tournament-Pack-20', 
                'Mystic-Fighters', '25th-Anniversary-Tin-Dueling-Heroes', '2020-Tin-of-Lost-Memories', '2022-Tin-of-the-Pharaohs-Gods', 
                '2021-Tin-of-Ancient-Battles', '2019-Gold-Sarcophagus-Tin', '2019-Gold-Sarcophagus-Tin-Mega-Pack', 
                'Legendary-Collection-25th-Anniversary-Edition', 'Breakers-of-Shadow', 'Hidden-Summoners', 'Structure-Deck-Zombie-Horde', 
                'Legendary-Collection-Kaiba-Mega-Pack', 'Legendary-Decks-II', 'Legendary-Collection-2-The-Duel-Academy-Years', 'OTS-Tournament-Pack-19', 
                'Return-of-the-Duelist', 'Rising-Rampage', 'Fists-of-the-Gadgets', 'Structure-Deck-Emperor-of-Darkness', 
                'Battle-Pack-2-War-of-the-Giants-Round-2', 'Ignition-Assault', 'Wing-Raiders', 'Structure-Deck-Spirit-Charmers',
                'OTS-Tournament-Pack-16', '2018-MegaTin-Mega-Pack', 'Maze-of-Millennia', 'Star-Pack-2013', 'Magicians-Force', '']
# expansions = []
cards = []
category = "Singles"
tcg = 'YuGiOh'
refresh_price = True

In [3]:
counter_threshold = 20

In [4]:
cardlist_file_path = f"{os.getcwd()}/data/{tcg}/cardlist.json"
updated_cards_prices_file_path = f"{os.getcwd()}/data/{tcg}/updated-cards-prices.json"

In [5]:
URL = f"https://www.cardmarket.com/en/{tcg}/Products"
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'
}

In [6]:
try:
    with open(cardlist_file_path, "r") as file:
        data = json.load(file)
except Exception as e:
    data = {}

In [7]:
def setup_driver():
    options = Options()
    # options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-blink-features=AutomationControlled')
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
    driver.set_window_size(random.randint(800, 1200), random.randint(600, 800))
    actions = ActionChains(driver)
    print("Driver set")
    
    # Accepting Cookies
    driver.get(URL)
    cookie_button = """//*[@id="CookiesConsent"]/div/div/form/div/button"""
    cookie_btn = driver.find_element(By.XPATH, cookie_button)
    random_delay(4, 6)
    try:
        cookie_btn.click()
        print("Cookie Accepted")
        random_delay(2, 3)
    except Exception as e:
        print(f'Could not click on cookie button {url}: {e}')
    return driver, actions

In [8]:
def random_delay(min_delay=3, max_delay=10):
    time.sleep(random.uniform(min_delay, max_delay))

In [9]:
driver, actions = setup_driver()

Driver set
Cookie Accepted


In [10]:
def save_data(data):
    sorted_data = {
        category: {
            exp_name: {
                card_name: card_data
                for card_name, card_data in sorted(exp_data.items())
            }
            for exp_name, exp_data in sorted(category_data.items())
        }
        for category, category_data in sorted(data.items())
    }
    with open(cardlist_file_path, "w") as file:
        json.dump(sorted_data, file, indent=4)

In [11]:
try:
    with open(updated_cards_prices_file_path, "r") as file:
        updated_cards_prices = json.load(file)
except Exception as e:
    updated_cards_prices = {}

In [12]:
def save_updated_cards_prices(updated_cards_prices):
    with open(updated_cards_prices_file_path, "w") as file:
        json.dump(updated_cards_prices, file, indent=4)

In [13]:
price_from = """//*[@id="tabContent-info"]/div/div[1]/div/div[2]/dl/dd[6]"""
price_trend = """//*[@id="tabContent-info"]/div/div[1]/div/div[2]/dl/dd[7]/span"""
price_30_day_avg = """//*[@id="tabContent-info"]/div/div[1]/div/div[2]/dl/dd[8]/span"""
pricing_basis = "price_trend"

In [14]:
def convert_currency_string(currency_str):
    if '£' in currency_str and '.' in currency_str and ',' in currency_str:
        currency_str = str(float(currency_str.strip('£').replace('.', '').replace(',', '.'))*1.16).replace('.', ',')
    elif '£' in currency_str:
        currency_str = str(float(currency_str.strip('£').replace(',', '.'))*1).replace('.', ',')
    cleaned_str = currency_str.strip('€ £')
    
    # If there's a comma and no period, it's likely a decimal comma
    if ',' in cleaned_str and '.' not in cleaned_str:
        cleaned_str = cleaned_str.replace(',', '.')
    else:
        # Remove periods used as thousand separators
        cleaned_str = cleaned_str.replace('.', '').replace(',', '.')

    return float(cleaned_str)

In [15]:
def get_price(data, category, selected_expansions = [], selected_cards = []):
    global driver, actions
    cards = []
    for expansion, card_list in data[category].items():
        for card_name, card_info in card_list.items():
            card_info_copy = card_info.copy()  # Avoid modifying the original data
            card_info_copy.update({'expansion': expansion, 'card_name': card_name})
            cards.append(card_info_copy)
    # Setting 'expansion' and 'card_name' as the index to make it easier to filter
    df = pd.DataFrame(cards).set_index(['expansion', 'card_name'])

    if selected_expansions or selected_cards:
        df = df.loc[df.index.get_level_values('expansion').isin(selected_expansions) | 
                    df.index.get_level_values('card_name').isin(selected_cards)].copy()
    # df = df[df['quantity'] > 0]

    counter = 0
    for index, row in df.iterrows():
        if counter == counter_threshold:
            save_data(data)
            save_updated_cards_prices(updated_cards_prices)
            driver.quit()
            driver, actions = setup_driver()
            counter = 0
        expansion, card_name = index
        if expansion in updated_cards_prices and card_name in updated_cards_prices[expansion]:
            continue
        print(f"Getting price for: {card_name}")
        url = f"{URL}/{category}/{expansion}/{card_name}"
        driver.get(url)
        random_delay(1, 3)
        # actions.move_by_offset(random.randint(0, 100), random.randint(0, 100)).perform()
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        random_delay(3, 4)
        
        try:
            p_from = convert_currency_string(driver.find_element(By.XPATH, price_from).text)
            df.at[index, 'price_from'] = p_from
            data[category][expansion][card_name]['price_from'] = p_from
        except Exception as e:
            print(f'Could not retrieve price_from for {url}: {e}')
        try:
            p_trend = convert_currency_string(driver.find_element(By.XPATH, price_trend).text)
            data[category][expansion][card_name]['price_trend'] = p_trend
        except Exception as e:
            print(f'Could not retrieve price_trend for {url}: {e}')
        try:
            p_30_day = convert_currency_string(driver.find_element(By.XPATH, price_30_day_avg).text)
            df.at[index, 'price_30_day_avg'] = p_30_day
            data[category][expansion][card_name]['price_30_day_avg'] = p_30_day
        except Exception as e:
            print(f'Could not retrieve price_30_day_avg for {url}: {e}')

        if expansion in updated_cards_prices:
            updated_cards_prices[expansion].append(card_name)
        else:
            updated_cards_prices[expansion] = [card_name]
        counter += 1
    save_data(data)
    save_updated_cards_prices(updated_cards_prices)

In [16]:
# print(convert_currency_string('£1.999,99'))
# print(convert_currency_string('£0,17'))
# print(convert_currency_string('£0.02'))
# print(convert_currency_string('0,317 €'))
# print(convert_currency_string('1.999,99  €'))

In [ ]:
if refresh_price:
    get_price(data, category, expansions, cards)

Getting price for: Fossil-Dig-V3-Secret-Rare
Getting price for: Galaxy-Eyes-Afterglow-Dragon-V1-Super-Rare
Getting price for: Galaxy-Eyes-Afterglow-Dragon-V2-Ultra-Rare
Getting price for: Galaxy-Eyes-Afterglow-Dragon-V3-Secret-Rare
Getting price for: Galaxy-Eyes-Full-Armor-Photon-Dragon-V1-Super-Rare
Getting price for: Galaxy-Eyes-Full-Armor-Photon-Dragon-V2-Ultra-Rare
Getting price for: Galaxy-Eyes-Full-Armor-Photon-Dragon-V3-Secret-Rare
Getting price for: Ghost-Belle-Haunted-Mansion-V1-Super-Rare
Getting price for: Ghost-Belle-Haunted-Mansion-V2-Ultra-Rare
Getting price for: Ghost-Belle-Haunted-Mansion-V3-Secret-Rare
Getting price for: Ghostrick-Angel-of-Mischief-V1-Super-Rare
Getting price for: Ghostrick-Angel-of-Mischief-V2-Ultra-Rare
Getting price for: Ghostrick-Angel-of-Mischief-V3-Secret-Rare
Getting price for: Harpies-Feather-Storm-V1-Super-Rare
Getting price for: Harpies-Feather-Storm-V2-Ultra-Rare
Getting price for: Harpies-Feather-Storm-V3-Secret-Rare
Getting price for: Hera

In [ ]:
def get_df(data, category, selected_expansions = [], selected_cards = []):
    cards = []
    for expansion, card_list in data[category].items():
        for card_name, card_info in card_list.items():
            card_info_copy = card_info.copy()  # Avoid modifying the original data
            card_info_copy.update({'expansion': expansion, 'card_name': card_name})
            cards.append(card_info_copy)

    df = pd.DataFrame(cards).set_index(['expansion', 'card_name'])

    if selected_expansions or selected_cards:
        df = df.loc[df.index.get_level_values('expansion').isin(selected_expansions) | 
                    df.index.get_level_values('card_name').isin(selected_cards)].copy()
    return df

In [ ]:
df = get_df(data, category, expansions, cards)

In [ ]:
df[(df['price_from'] > df['price_trend']*1.2) & (df['quantity'] > 0)]

In [ ]:
def get_worth(data, category, selected_expansions = [], selected_cards = []):
    cards = []
    for expansion, card_list in data[category].items():
        for card_name, card_info in card_list.items():
            card_info_copy = card_info.copy()  # Avoid modifying the original data
            card_info_copy.update({'expansion': expansion, 'card_name': card_name})
            cards.append(card_info_copy)

    df = pd.DataFrame(cards).set_index(['expansion', 'card_name'])

    if selected_expansions or selected_cards:
        df = df.loc[df.index.get_level_values('expansion').isin(selected_expansions) | 
                    df.index.get_level_values('card_name').isin(selected_cards)].copy()
    df = df[df['quantity'] > 0]
    
    df['total_price'] = df['quantity'] * df[pricing_basis]
    print(df.reset_index()[['card_name', 'quantity', pricing_basis, 'total_price']])
    sum = df['total_price'].sum()
    print(f"{sum:.2f}")

In [ ]:
get_worth(data, category, expansions, cards)

In [ ]:
driver.quit()